In [1]:
from datasets import load_dataset, Audio
import pandas as pd
import warnings
import random
import requests
import gzip
from collections import Counter
import numpy as np
import io
import numpy.core.multiarray as multi
from sklearn.metrics import classification_report, balanced_accuracy_score
import matplotlib.pyplot as plt
from sklearn import datasets, metrics, svm
from sklearn.model_selection import train_test_split
import soundfile as sf
import gzip
import torch
import matplotlib.pyplot as plt
from IPython.display import Audio as iAudio
from npc_gzip.compressors.bz2_compressor import Bz2Compressor
from npc_gzip.compressors.gzip_compressor import GZipCompressor
from npc_gzip.knn_classifier import KnnClassifier

In [2]:
def get_data(dataset) -> tuple:
    """
    Pulls the Project CETI sperm whale vocalizations
    training data and the second being the test
    data. Each tuple contains the audio and label
    respectively as numpy arrays.

    """

    train_iter = dataset['train']
    test_iter = dataset['test']

    train_audio =  [audio_array['bytes'] for audio_array in train_iter['audio']]
    train_labels = train_iter['coda_type']
    
    test_audio  = [audio_array['bytes'] for audio_array in test_iter['audio']]
    test_labels = test_iter['coda_type']
    
    train_audio = np.array(train_audio)
    train_labels = np.array(train_labels)

    test_audio = np.array(test_audio)
    test_labels = np.array(test_labels)

    train = (train_audio, train_labels)
    test = (test_audio, test_labels)

    return (train, test)

In [3]:
def fit_model(train_audio: np.ndarray, train_labels: np.ndarray, distance_metric: str = "ncd") -> KnnClassifier:
    """
    Fits a Knn-GZip compressor on the train
    data and returns it.

    Arguments:
        train_text (np.ndarray): Training dataset as a numpy array.
        train_labels (np.ndarray): Training labels as a numpy array.

    Returns:
        KnnClassifier: Trained Knn-Compressor model ready to make predictions.
    """
    compressor: BaseCompressor = Bz2Compressor()
    model: KnnClassifier = KnnClassifier(
        compressor=compressor,
        training_inputs=train_audio,
        training_labels=train_labels,
        distance_metric=distance_metric,
    )

    return model

In [4]:
dataset = load_dataset("autumnjohnson/ceti_audio").cast_column("audio", Audio(decode=False, sampling_rate=16000))

In [5]:
((train_audio, train_labels), (test_audio, test_labels)) = get_data(dataset)

## Train model

In [6]:
model = fit_model(train_audio, train_labels)

## Initialize test data

In [7]:
random_indicies = np.random.choice(test_audio.shape[0], len(test_audio), replace=False)

In [8]:
sample_test_text = test_audio[random_indicies]

In [9]:
sample_test_labels = test_labels[random_indicies]

## Generate predictions for test set

In [ ]:
 # Here we use the `sampling_percentage` to save time
 # at the expense of worse predictions. This
 # `sampling_percentage` selects a random % of training
 # data to compare `sample_test_text` against rather
 # than comparing it against the entire training dataset.
top_k = 1
(distances, labels, similar_samples) = model.predict(sample_test_text, top_k, sampling_percentage=0.01)

Compressing input...:  21%|██        | 75/353 [00:24<01:37,  2.86it/s]

In [ ]:
flattened_labels = labels.flatten()

In [ ]:
p = Counter(flattened_labels)  
l = Counter(sample_test_labels)
x =(l['1+1+3']+p['1+1+3'])
l

In [ ]:
print(classification_report(sample_test_labels, flattened_labels.reshape(-1)))

In [ ]:
disp = metrics.ConfusionMatrixDisplay.from_predictions(sample_test_labels, flattened_labels.reshape(-1))
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

In [ ]:
balanced_accuracy_score(sample_test_labels, flattened_labels.reshape(-1))